In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pathlib 
import sys
#from tqdm import tqdm

sys.path.append("..")
from utils.utils import show_mask_image

from  utils.sample_patches_main import sample_patches, sample_hard_negatives, sample_blob_negatives

In [ ]:
dataset_root = pathlib.Path(dataset_root)
images_dir = dataset_root / 'images'
masks_dir = dataset_root / 'masks'
bounding_boxes = pd.read_csv(self.dataset_root / 'bounding_boxes.csv')

        bounding_boxes.groupby("id")        



NameError: name 'Dataset' is not defined

In [20]:
class groupby_data(cbis_ddsm):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.dataset_root = get_parameter(self.config, ['DataModule', 'dataset_root'])

NameError: name 'cbis_ddsm' is not defined